In [ ]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns

# Importación de archivos

In [ ]:
reporte = pd.read_csv('ReporteDetalladoSms.csv',sep=',')
reporte = pd.DataFrame(reporte)

sms = pd.read_excel('SMS Comerciales Informativos.xlsx')
sms = pd.DataFrame(sms)

email = pd.read_excel('email.xlsx')
email = pd.DataFrame(email)

# Manipulación de datos para ReporteDetallado

In [ ]:
# Filtro según guiones para SMS >= $100.000

mayores = reporte.loc[(reporte['Valor'] >= 100000.0)]
conditions1 = [(mayores.Mensaje.str.contains('Usted realizo Retiro') | 
                mayores.Mensaje.str.contains('Usted realizo PSE - Pago productos') |
                mayores.Mensaje.str.contains('Usted realizo CNB - Retiro') | 
                mayores.Mensaje.str.contains('Usted realizo Compra por') |
                mayores.Mensaje.str.contains('Usted realizo Transferencia') |
                mayores.Mensaje.str.contains('Transfiya')
                )]
choicelist1 = ['NO']
mayores['Facturable'] = np.select(conditions1, choicelist1, default='SI')

# Filtro según guiones para SMS < $100.000 e Informativos/Promocionales

menores = reporte.loc[(reporte['Valor'] < 100000.0) | (pd.isna(reporte['Valor']) == True )]
conditions2 = [(menores.Mensaje.str.contains('OTP') | 
                menores.Mensaje.str.contains('Usted ha ingresado exitosamente') |
                menores.Mensaje.str.contains('Usted se ha registrado exitosamente') |              
                menores.Mensaje.str.contains('Codigo') |
                menores.Mensaje.str.contains('codigo') |
                menores.Mensaje.str.contains('Código') |
                menores.Mensaje.str.contains('código') |
                menores.Mensaje.str.contains('PIN') |
                menores.Mensaje.str.contains('le ha remitido de manera fisica') |
                menores.Mensaje.str.contains('Su usuario es')
                )]
choicelist2 = ['NO']
menores['Facturable'] = np.select(conditions2, choicelist2, default='SI')

# Concatenación de menores y mayores

full = pd.concat([mayores, menores])

# Filtro por mensajes que serán Facturables o No Facturables

facturables_reporte = full[full['Facturable'] == 'SI']
nofacturables_reporte = full[full['Facturable'] == 'NO']

# Cálculos para cobro de mensajes por entidad

facturables_reporte['Número de mensajes'] = 1
facturables_reporte.rename(columns={'Número de mensajes': 'Cantidad SMS'}, inplace=True)
facturables_reporte['Valor Unitario'] = 10
facturables_reporte['Total'] = facturables_reporte['Cantidad SMS'] * facturables_reporte['Valor Unitario']
facturables_reporte['Iva'] = facturables_reporte['Total'] * 0.19
facturables_reporte['Total + Iva'] = facturables_reporte['Total'] + facturables_reporte['Iva']
facturables_reporte = facturables_reporte.groupby(['Entidad', 'Valor Unitario']).agg({'Cantidad SMS':['count'], 'Total':['sum'], 'Iva':['sum'], 'Total + Iva':['sum']}).reset_index()
facturables_reporte = facturables_reporte[['Entidad','Cantidad SMS','Valor Unitario','Total','Iva','Total + Iva']]

# Manipulación de datos para SMS

In [ ]:
# Cálculos para cobro de mensajes por entidad

sms['Valor Unitario'] = 10
sms['Total'] = sms['Cantidad SMS'] * sms['Valor Unitario']
sms['Iva'] = sms['Total'] * 0.19
sms['Total + Iva'] = sms['Total'] + sms['Iva']

# Manipulación de datos para Email

In [ ]:
# Cálculos para cobro de emails por entidad

email['Valor Unitario'] = 5
email['Total'] = email['Cantidad email'] * email['Valor Unitario']
email['Iva'] = email['Total'] * 0.19
email['Total + Iva'] = email['Total'] + email['Iva']

# Exportación de Informes a Excel

In [ ]:
# Exportación de cada uno de los Informes a hojas independientes del archivo Excel

with pd.ExcelWriter('Informe Facturación.xlsx') as writer:
    facturables_reporte.to_excel(writer, sheet_name='ReporteDetallado')
    sms.to_excel(writer, sheet_name='SMS Comerciales Informativos')
    email.to_excel(writer, sheet_name='Email')